# I. Initial Imports (more found throughout workbook)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from itertools import combinations
import explore

from acquire import get_zillow_data
from wrangle import wrangle_zillow
from wrangle import scale_zillow
from splitter import splitter

import warnings
warnings.filterwarnings("ignore")

# II. Load Data

## Step 1 - Dataset analyzed in SQL
- Decided on features to use from properties_2017 table based on initial analysis (and hypothesis) that the following numerical features are important: Year built, number of bathrooms and bedrooms, building quality type (on a scale from 1-12, but continuous), calculated finsihed square feet, lot size in square feet, and the number of fireplaces, pools and garages.  Selected one categorical feature, fips.  
- Right joined with predictions_2017 on 'parcelid'
- See acquire.py for SQL query used (to include features selected)

In [2]:
df = get_zillow_data()
df.sample(5)

,bedrooms,bathrooms,sqft,valuation,yearbuilt,quality,fireplaces,lotsize,pools,garages,fips
46501,5.0,4.0,4510.0,2546378.0,2012.0,11.0,NaN,7886.0,NaN,NaN,6037.0
1749,4.0,2.0,1835.0,62048.0,1964.0,NaN,1.0,6500.0,NaN,2.0,6111.0
19618,3.0,2.0,2263.0,337404.0,1972.0,NaN,NaN,7280.0,NaN,2.0,6059.0
21546,4.0,4.0,1890.0,601100.0,2004.0,7.0,NaN,2499.0,NaN,NaN,6037.0
15950,3.0,1.0,1066.0,389610.0,1950.0,4.0,NaN,6626.0,NaN,NaN,6037.0


## Step 2 - Examine data to determine how to prep/clean (for wrangle.py file)

In [ ]:
df.info()

##### Check for dupliactes:

In [3]:
df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52442 entries, 0 to 52441
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bedrooms    52442 non-null  float64
 1   bathrooms   52442 non-null  float64
 2   sqft        52360 non-null  float64
 3   valuation   52441 non-null  float64
 4   yearbuilt   52326 non-null  float64
 5   quality     33741 non-null  float64
 6   fireplaces  7243 non-null   float64
 7   lotsize     52073 non-null  float64
 8   pools       11096 non-null  float64
 9   garages     18015 non-null  float64
 10  fips        52442 non-null  float64
dtypes: float64(11)
memory usage: 4.8 MB


##### Run value counts on all columns:

In [25]:
for i in df.columns:
    print(i)
    print(df[i].value_counts())
    print(f'{df[df[i].isnull()].shape[0]} null values')
    print('\n-----\n')

bedrooms
3.0    23186
4.0    14994
2.0     8211
5.0     3774
1.0      543
6.0      540
Name: bedrooms, dtype: int64
0 null values

-----

bathrooms
2.0    21758
3.0    10485
1.0     9438
2.5     3917
4.0     2106
3.5      892
1.5      822
5.0      713
4.5      645
6.0      245
5.5      198
6.5       29
Name: bathrooms, dtype: int64
0 null values

-----

sqft
1120.0    127
1200.0    118
1080.0     93
936.0      86
1176.0     83
         ... 
3707.0      1
4591.0      1
4215.0      1
3559.0      1
5350.0      1
Name: sqft, Length: 4388, dtype: int64
0 null values

-----

valuation
455000.0    34
600000.0    30
500000.0    24
450000.0    24
550000.0    24
            ..
977685.0     1
348301.0     1
502027.0     1
455180.0     1
49546.0      1
Name: valuation, Length: 37998, dtype: int64
0 null values

-----

yearbuilt
1955.0    1889
1950.0    1594
1954.0    1511
1953.0    1480
1956.0    1373
          ... 
1892.0       1
1878.0       1
1882.0       1
1894.0       1
1880.0       1
Name: y

##### After examination of the value counts, my initial cleaning consists of the following:
1. It seems that fireplaces, pools and garages are all NaN if they have none.  Therefore, I will replace all their NaNs with a zero.
2. I need to investigate the quality score more, as it has a very large number of Nulls.  Quality is a metric from 1 (best) to 12 (worst).  Is there a way to fill in those numbers based on other features?

In [4]:
df['fireplaces'] = df.fireplaces.fillna(value=0)
df['pools'] = df.pools.fillna(value=0)
df['garages'] = df.garages.fillna(value=0)
df.head(2)

,bedrooms,bathrooms,sqft,valuation,yearbuilt,quality,fireplaces,lotsize,pools,garages,fips
0,4.0,3.5,3100.0,1023282.0,1998.0,NaN,0.0,4506.0,0.0,2.0,6059.0
1,2.0,1.0,1465.0,464000.0,1967.0,NaN,1.0,12647.0,0.0,1.0,6111.0


##### Quality Investigation:

In [ ]:
# Create dataframe with no nulls for quality analysis
df_qual = df.dropna()

In [ ]:
df_qual.quality.hist()

Since the quality graph is somewhat normal, I think I can use a linear model to fill those values.

In [ ]:
sns.boxplot(data=df_qual, x='fips_name', y='quality')

Aha, is only good for Los Angeles.

In [ ]:
X = df_qual.drop(columns=['quality'])
y = df_qual.quality

In [ ]:
from sklearn.linear_model import LinearRegression

ols_model = LinearRegression().fit(X,y)

df_qual['yhat'] = ols_model.predict(X)

df_qual.head(3)

In [ ]:
import evaluate
yhat = df_qual.yhat

In [ ]:
evaluate.better_than_baseline(y,yhat)

In [ ]:
df.info()

In [ ]:
quality = df.quality
quality

In [ ]:
df = df.drop(columns = 'quality')

In [ ]:
df = df.dropna()
df['quality'] = quality
df.info()

In [ ]:
X = df.drop(columns=['quality'])
df['yhat'] = ols_model.predict(X)

In [ ]:
df['quality'] = df.quality.fillna(value=df.yhat)
df

In [ ]:
df.quality.max(), df.quality.min()

In [ ]:
#plt.hist(y_validate.G3, color='blue', alpha=.5, label="Actual Final Grades")
plt.figure(figsize = (12,6))
plt.hist(df.quality, color='blue', alpha = .5, label = 'Quality', bins=24)
plt.hist(df.yhat, color='orange',alpha = .25, label = 'yhat', bins=24)
plt.legend
plt.show()

In [ ]:
df[df['quality']<2]

In [ ]:
df[df['quality']>12]

##### After examining the quality scores, I feel they may be too subjective to be correlated to other features, even with the linear regression model having a higher baseline RMSE (although it was fairly minor).  Looking at the extreme values, there seems to be nothing tha sticks out about either end (<2, >12) other than most of the >12 are actually huge houses with high doller values.

##### Therefore, I will drop quality from my main df examination, but plan on running a seperate analysis on only those who have a quality score as well as the other features being examined.

In [ ]:
df.describe().T

In [ ]:
for col in df.columns:
    print(col)
    df[col].hist()
    plt.show()
    print(100*df[col].value_counts()/df.shape[0])
    print('\n-----\n')

In [ ]:
df[df['lotsize']>217800]

##### I used the following general line of code to examine each feature in details to determine where to cut values off: df[df[feature]>n].sort_values(by=feature)

##### From this I determined the following to remove some of the larger outliers, to be done in wrangle:
1. Set range of bedrooms and bathrooms from 1-6 (0<b<7)
2. Remove all listings with a home size greater than 10,000 sqft and less than 400 sqft
3. Remove all listings with more than 4 garages (g<5)
4. Remove all listing with more than 3 fireplaces (f<4)
5. Remove listings with a valuation greater than \$10MM or less than 10k
6. Remove all listings with a lot size greater than 217,800 sqft (5 acres)
7. Also remove all listings with a lot size less than 1/2 the sqft of the home.

In [5]:
df = df.drop(columns='quality')
df = df.dropna()

In [6]:
df = df[df['bathrooms'] > 0]
df = df[df['bedrooms'] > 0]
df = df[df['bathrooms'] < 7]
df = df[df['bedrooms'] < 7]
df = df[df['sqft'] > 400]
df = df[df['sqft'] < 10000]
df = df[df['valuation'] > 10000]
df = df[df['valuation'] < 5000000]
df = df[df['fireplaces'] < 4]
df = df[df['garages'] < 5]
df = df[df['lotsize'] < 217800]
df = df[df['lotsize'] > (.5 * df['sqft'])]
df.shape[0]/51960

0.9862971516551193

This leaves ~98.6% of the records for exploration and modeling.

In addition to removing these listings, I will also convert floats to integers, where applicable, and rename the fips with their actual names.

In [7]:
df['fips_name'] = np.where(df.fips == 6037, 'Los Angeles', np.where(df.fips == 6059, 'Orange','Ventura') )
df = df.drop(columns = 'fips')

# III. Univariate Plots and Initial Hypotheses

In [ ]:
for col in df.columns:
    print(col)
    sns.boxplot(y=df[col].values)
    plt.show()
    print('\n-----\n')

In [ ]:
sns.histplot(x='fips_name',data=df)
plt.show()

## Initial Hypotheses
1. Having a fireplace(s), garage(s) and/or pool increases home value.
2. Larger lot size increases home value.
3. Larger square foot size increases home value.
4. More bedrooms and bathrooms increase home value.
5. There is a difference in home values due to geography.

# IV. Bivariate and Multivariate EDA

#### First, we must split the data.  I want to ensure the categorical variable, fips name, is equally distributed across the splits, so I will stratify on fips_name.

In [8]:
train, validate, test = splitter(df, target='fips_name')
train.head(2)

Train = 28698 rows (56.0%) | Validate = 12300 rows (24.0%) | Test = 10250 rows (20.0%)


,bedrooms,bathrooms,sqft,valuation,yearbuilt,fireplaces,lotsize,pools,garages,fips_name
24401,3.0,2.0,1163.0,505000.0,2000.0,1.0,3757.0,0.0,1.0,Ventura
11376,4.0,3.0,2633.0,420576.0,1994.0,0.0,8000.0,0.0,2.0,Orange


Plot all variable pairs to look for correlations.

In [ ]:
explore.plot_variable_pairs(train)

In [ ]:
explore.plot_categorical_and_continuous_vars(train)

Plot all variables against target

In [ ]:
num_cols = [col for col in train.columns if train[col].dtype != 'object']

for i in num_cols:
    sns.lmplot(data = train, x=i, y='valuation', col = 'fips_name')
    plt.show()

##### Looking at the charts, I see that there are no garages or fireplaces for any of the Los Angeles homes.  Since LA is the majority of the dataset, I will remove these features as unimportant.  Thinking about statistical tests:
1. Homes with pools have a higher valuation than homes without. (t-test)
2. Orange county has the highest valuation, followed by Ventura and then LA (mutliple t-tests)
3. More bathrooms correlated with higher valuation (pearsons)
4. More bedrooms correlated with higher valuation (pearsons)
5. Higher square footage correlated with higher valuation (pearsons)
6. Larger lot size correlated with higher valuations (pearsons)

## Hypotheses testing

In [ ]:
import scipy.stats as stats
alpha = .05

#### 1. Do homes with pools have a higher valuation than homes without? (one-sided, independent t-test)

$H_{0}$ = Homes with pools have the same or lower valuations than homes without.

$H_{1}$ = Homes with pools have a higher valuation than homes without.

Assumptions: (X)Independent | (X) Normal (or 30+ obs) | (?) Equal variances?

In [ ]:
print(train[train['pools']==1].valuation.var())
print(train[train['pools']==0].valuation.var())

*Must set equal_var to False*

In [ ]:
t, p = stats.ttest_ind(train[train['pools']==1].valuation, train[train['pools']==0].valuation, equal_var=False)
t, p / 2

In [ ]:
print("Is t < 0? ", t < 0)
print("is p/2 < alpha? ", p / 2 < alpha)

We REJECT the null hypothesis that homes with pools have the same or lower valuations than homes without. 

#### 2. Does Orange county have the highest mean valuations followed by Ventura and then LA? (mutliple one sided, independent t-tests)
$H_{0}$ = X-fip has the same or lower valuations than Y-fip without.

$H_{1}$ = X-fip has a higher valuation than Y-fip without.

where X1 is Orange, Y1 is Ventura, X2 is Ventura and Y2 is Los Angeles

Assumptions: (X)Independent | (X) Normal (or 30+ obs) | (?) Equal variances?

In [ ]:
print(train[train['fips_name']=='Orange'].valuation.var())
print(train[train['fips_name']=='Ventura'].valuation.var())
print(train[train['fips_name']=='Los Angeles'].valuation.var())

*While Orange and LA have equal variances, I'll go ahead and set to False when comparing to Ventura*

1. Orange vs Ventura

In [ ]:
t, p = stats.ttest_ind(train[train['fips_name']=='Orange'].valuation, train[train['fips_name']=='Ventura'].valuation, equal_var=False)
t, p / 2

We REJECT the null hypothesis that Orange has the same or lower valuations than Ventura.

In [ ]:
t, p = stats.ttest_ind(train[train['fips_name']=='Ventura'].valuation, train[train['fips_name']=='Los Angeles'].valuation, equal_var=False)
t, p / 2

We REJECT the null hypothesis that Ventura has the same or lower valuations than Los Angeles.

#### 3. More bathrooms correlated with valuation (pearsons)
$H_{0}$ = Bathrooms are not correlated with valuation.

$H_{1}$ = Bathrooms are correlated with valuation.

In [ ]:
corr, p = stats.pearsonr(train.bathrooms, train.valuation)
corr, p

We REJECT the null hypothesis that bathrooms are not correlated with valuation.

#### 4. More bedrooms correlated with higher valuation (pearsons)
$H_{0}$ = Bedrooms are not correlated with valuation.

$H_{1}$ = Bedrooms are correlated with valuation.

In [ ]:
corr, p = stats.pearsonr(train.bedrooms, train.valuation)
corr, p

We REJECT the null hypothesis that bathrooms are not correlated with valuation.

#### 5. Higher square footage correlated with higher valuation (pearsons)
$H_{0}$ = Square footage of home is not correlated with valuation.

$H_{1}$ = Square footage of home is correlated with valuation.

In [ ]:
corr, p = stats.pearsonr(train.sqft, train.valuation)
corr, p

We REJECT the null hypothesis that square footage of him is not correlated with valuation.

#### 6. Larger lot size correlated with higher valuations (pearsons)
$H_{0}$ = Lot size is not correlated with valuation.

$H_{1}$ = Lot size is correlated with valuation.

In [ ]:
corr, p = stats.pearsonr(train.lotsize, train.valuation)
corr, p

We REJECT the null hypothesis that lot size is not correlated with valuation.

### *Sidequest: Quality Investigation EDA*

##### Quick EDA examining Quality:

In [ ]:
df_qual = df_qual.drop(columns = ['fireplaces','bathrooms','garages'])
df_qual.info()

In [ ]:
df_qual = df_qual.dropna()

In [ ]:
df_qual['fips_name'] = np.where(df_qual.fips == 6037, 'Los Angeles', np.where(df_qual.fips == 6059, 'Orange','Ventura') )
df_qual = df_qual.drop(columns = 'fips')

In [ ]:
df_qual = df_qual[df_qual['bedrooms'] > 0]
df_qual = df_qual[df_qual['bedrooms'] < 7]
df_qual = df_qual[df_qual['sqft'] > 400]
df_qual = df_qual[df_qual['sqft'] < 10000]
df_qual = df_qual[df_qual['valuation'] > 10000]
df_qual = df_qual[df_qual['valuation'] < 5000000]
df_qual = df_qual[df_qual['lotsize'] < 217800]
df_qual = df_qual[df_qual['lotsize'] > (.5 * df_qual['sqft'])]
df_qual.shape[0]/33434

In [ ]:
train_qual, validate_qual, test_qual = splitter(df_qual, target='fips_name')

In [ ]:
sns.barplot(data = train_qual, x='quality', y='valuation')

In [ ]:
corr, p = stats.pearsonr(train_qual.quality, train_qual.valuation)
corr, p

In [ ]:
train_qual.fips_name.value_counts()

## EDA Outcomes:

1. Orange county is associated with the highest valuation, then Ventura then LA.
2. Bathrooms correlated with square feet, adn sqft better corelated with valiuation so dropping bathrooms.
3. Quality seems well correlated with valuations, and the metrics seems wrong (higher seems better), so worth more investigation.
4. Bedrooms are relevant so keep them
5. Pools are so keep them
6. Lot size and pool correlated with higher valuations, albeit not as strongly.
7. Dropping garages and fireplaces.

# V. Feature Engineering

In [ ]:
train.info()

In [9]:
train = train.astype({'bedrooms':'int', 'sqft':'int', 'valuation':'int', 'yearbuilt':'int','fireplaces':'int','lotsize':'int','pools':'int','garages':'int'})

In [10]:
columns_to_remove = ['bathrooms','fireplaces','garages']
train = train.drop(columns = columns_to_remove)
train

,bedrooms,sqft,valuation,yearbuilt,lotsize,pools,fips_name
24401,3,1163,505000,2000,3757,0,Ventura
11376,4,2633,420576,1994,8000,0,Orange
41985,3,1401,320607,1937,5963,1,Los Angeles
1184,5,3762,1057082,1980,20366,1,Ventura
8066,5,3170,777109,1979,10000,1,Orange
...,...,...,...,...,...,...,...
38232,3,939,350260,1962,4997,0,Los Angeles
6148,3,1825,334507,1964,7800,0,Orange
45700,3,2084,268418,1969,7200,1,Orange
38230,5,5540,2979716,1928,37133,1,Los Angeles


In [11]:
validate = validate.astype({'bedrooms':'int', 'sqft':'int', 'valuation':'int', 'yearbuilt':'int','fireplaces':'int','lotsize':'int','pools':'int','garages':'int'})
test = test.astype({'bedrooms':'int', 'sqft':'int', 'valuation':'int', 'yearbuilt':'int','fireplaces':'int','lotsize':'int','pools':'int','garages':'int'})

In [12]:
validate = validate.drop(columns = columns_to_remove)
test = test.drop(columns = columns_to_remove)

In [13]:
train_scaled, validate_scaled, test_scaled = scale_zillow(train, validate, test)
train_scaled.sample(10)

,bedrooms,sqft,yearbuilt,lotsize,pools,fips_name,valuation,fips_name_Los Angeles,fips_name_Orange,fips_name_Ventura
11614,0.4,0.072978,0.566176,0.030479,0.0,Orange,431697,0,1,0
24479,0.4,0.090544,0.514706,0.033159,0.0,Los Angeles,84959,1,0,0
35314,0.4,0.174040,0.242647,0.031074,0.0,Los Angeles,693683,1,0,0
1237,0.4,0.144546,0.882353,0.039603,0.0,Los Angeles,494867,1,0,0
35388,0.4,0.227825,0.838235,0.017195,0.0,Orange,767135,0,1,0
40452,0.4,0.175667,0.875000,0.032850,0.0,Los Angeles,538081,1,0,0
30731,0.6,0.244632,0.639706,0.029358,0.0,Orange,878133,0,1,0
17585,0.4,0.141184,0.610294,0.682753,0.0,Los Angeles,328921,1,0,0
36227,0.6,0.097918,0.514706,0.025092,0.0,Los Angeles,445278,1,0,0
30602,0.4,0.093038,0.595588,0.026010,0.0,Orange,440330,0,1,0


In [14]:
X_train = train_scaled.drop(columns=['fips_name','valuation'])
y_train = train_scaled.valuation

X_validate = validate_scaled.drop(columns=['fips_name','valuation'])
y_validate = validate_scaled.valuation

X_test = test_scaled.drop(columns=['fips_name','valuation'])
y_test = test_scaled.valuation

In [15]:
from sklearn.feature_selection import SelectKBest, f_regression

for i in range(1,9):
    # parameters: f_regression stats test, give me 8 features
    f_selector = SelectKBest(f_regression, k=i)

    # find the top 8 X's correlated with y
    f_selector.fit(X_train, y_train)

    # boolean mask of whether the column was selected or not. 
    feature_mask = f_selector.get_support()

    # get list of top K features. 
    f_feature = X_train.iloc[:,feature_mask].columns.tolist()
    print(f_feature)

['sqft']
['bedrooms', 'sqft']
['bedrooms', 'sqft', 'pools']
['bedrooms', 'sqft', 'yearbuilt', 'pools']
['bedrooms', 'sqft', 'yearbuilt', 'lotsize', 'pools']
['bedrooms', 'sqft', 'yearbuilt', 'lotsize', 'pools', 'fips_name_Orange']
['bedrooms', 'sqft', 'yearbuilt', 'lotsize', 'pools', 'fips_name_Los Angeles', 'fips_name_Orange']
['bedrooms', 'sqft', 'yearbuilt', 'lotsize', 'pools', 'fips_name_Los Angeles', 'fips_name_Orange', 'fips_name_Ventura']


In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE, RFECV

for i in range(1,9):
# initialize the ML algorithm
    lm = LinearRegression()

    # create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
    rfe = RFE(lm, n_features_to_select=i)

    # fit the data using RFE
    rfe.fit(X_train,y_train)  

    # get the mask of the columns selected
    feature_mask = rfe.support_

    # get list of the column names. 
    rfe_feature = X_train.iloc[:,feature_mask].columns.tolist()
    print(rfe_feature)

rfecv = RFECV(lm, min_features_to_select = 1)
rfecv.fit(X_train, y_train)
feature_mask = rfecv.support_
rfecv_feature = X_train.iloc[:,feature_mask].columns.tolist()
print(rfecv_feature)

['sqft']
['sqft', 'fips_name_Ventura']
['sqft', 'fips_name_Orange', 'fips_name_Ventura']
['sqft', 'fips_name_Los Angeles', 'fips_name_Orange', 'fips_name_Ventura']
['bedrooms', 'sqft', 'fips_name_Los Angeles', 'fips_name_Orange', 'fips_name_Ventura']
['bedrooms', 'sqft', 'lotsize', 'fips_name_Los Angeles', 'fips_name_Orange', 'fips_name_Ventura']
['bedrooms', 'sqft', 'yearbuilt', 'lotsize', 'fips_name_Los Angeles', 'fips_name_Orange', 'fips_name_Ventura']
['bedrooms', 'sqft', 'yearbuilt', 'lotsize', 'pools', 'fips_name_Los Angeles', 'fips_name_Orange', 'fips_name_Ventura']
['bedrooms', 'sqft', 'yearbuilt', 'lotsize', 'pools', 'fips_name_Los Angeles', 'fips_name_Orange', 'fips_name_Ventura']


##### I performed both a SelecKBest and RFE.  RFE ran without issue and is considered the superiros one, so went with it.
##### Then used an RFE Cross Validation funtion to determine optial number of features, and it looks like we should model using all of the features .

# VI. Modeling

#### BASELINE:

In [17]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

In [18]:
# We need y_train and y_validate to be dataframes to append the new columns with predicted values. 
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)

# 1. Predict target_pred_mean
valuation_pred_mean = y_train['valuation'].mean()
y_train['valuation_pred_mean'] = valuation_pred_mean
y_validate['valuation_pred_mean'] = valuation_pred_mean

# 2. compute target_pred_median
valuation_pred_median = y_train['valuation'].median()
y_train['valuation_pred_median'] = valuation_pred_median
y_validate['valuation_pred_median'] = valuation_pred_median

# 3. RMSE of target_pred_mean
rmse_train_mean = mean_squared_error(y_train.valuation, y_train.valuation_pred_mean)**(1/2)
rmse_validate_mean = mean_squared_error(y_validate.valuation, y_validate.valuation_pred_mean)**(1/2)

print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train_mean, 5), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate_mean, 5))

# 4. RMSE of G3_pred_median
rmse_train_median = mean_squared_error(y_train.valuation, y_train.valuation_pred_median)**(1/2)
rmse_validate_median = mean_squared_error(y_validate.valuation, y_validate.valuation_pred_median)**(1/2)

print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train_median, 5), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate_median, 5))

RMSE using Mean
Train/In-Sample:  495485.97678 
Validate/Out-of-Sample:  465856.70737
RMSE using Median
Train/In-Sample:  510083.1964 
Validate/Out-of-Sample:  479220.83355


#### OLS:

In [19]:
# create the model object
lm = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm.fit(X_train, y_train.valuation)

# predict train
y_train['valuation_pred_lm'] = lm.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.valuation, y_train.valuation_pred_lm)**(1/2)

# predict validate
y_validate['valuation_pred_lm'] = lm.predict(X_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.valuation, y_validate.valuation_pred_lm)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

RMSE for OLS using LinearRegression
Training/In-Sample:  384906.39842239546 
Validation/Out-of-Sample:  369566.43744094815


#### LASSOLARS:

In [20]:
# create the model object
lars = LassoLars(alpha=2.0)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lars.fit(X_train, y_train.valuation)

# predict train
y_train['valuation_pred_lars'] = lars.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.valuation, y_train.valuation_pred_lars)**(1/2)

# predict validate
y_validate['valuation_pred_lars'] = lars.predict(X_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.valuation, y_validate.valuation_pred_lars)**(1/2)

print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

RMSE for Lasso + Lars
Training/In-Sample:  384900.1347973966 
Validation/Out-of-Sample:  369482.21796410286


#### TWEEDIE

In [21]:
# create the model object
glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
glm.fit(X_train, y_train.valuation)

# predict train
y_train['valuation_pred_glm'] = glm.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.valuation, y_train.valuation_pred_glm)**(1/2)

# predict validate
y_validate['valuation_pred_glm'] = glm.predict(X_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.valuation, y_validate.valuation_pred_glm)**(1/2)

print("RMSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

RMSE for GLM using Tweedie, power=1 & alpha=0
Training/In-Sample:  416065.37577210896 
Validation/Out-of-Sample:  392293.512460416


#### POLYNOMIAL

In [22]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree=3)

# fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(X_train)

# transform X_validate_scaled & X_test_scaled
X_validate_degree2 = pf.transform(X_validate)
X_test_degree2 = pf.transform(X_test)

In [23]:
# create the model object
lm2 = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm2.fit(X_train_degree2, y_train.valuation)

# predict train
y_train['valuation_pred_lm2'] = lm2.predict(X_train_degree2)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.valuation, y_train.valuation_pred_lm2)**(1/2)

# predict validate
y_validate['valuation_pred_lm2'] = lm2.predict(X_validate_degree2)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.valuation, y_validate.valuation_pred_lm2)**(1/2)

print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

RMSE for Polynomial Model, degrees=2
Training/In-Sample:  370900.6881775274 
Validation/Out-of-Sample:  360923.02494560473


#### Polynomial Regression of 3 is the best

# VII. TEST

In [24]:
y_test = pd.DataFrame(y_test)

# predict on test
y_test['valuation_pred_lm'] = lm2.predict(X_test_degree2)

# evaluate: rmse
rmse_test = mean_squared_error(y_test.valuation, y_test.valuation_pred_lm)**(1/2)

print("RMSE for OLS Model using LinearRegression\nOut-of-Sample Performance: ", rmse_test)

RMSE for OLS Model using LinearRegression
Out-of-Sample Performance:  379421.06253291364


### Sidequest - Adding in quality

In [ ]:
train_qual.info()

In [ ]:
train_qual.drop(columns='fips_name')
validate_qual.drop(columns='fips_name')
test_qual.drop(columns='fips_name')

In [ ]:
train_qual = train_qual.astype({'bedrooms':'int', 'sqft':'int', 'valuation':'int', 'yearbuilt':'int','lotsize':'int','pools':'int','quality':'int'})
validate_qual = validate_qual.astype({'bedrooms':'int', 'sqft':'int', 'valuation':'int', 'yearbuilt':'int','lotsize':'int','pools':'int','quality':'int'})
test_qual = test_qual.astype({'bedrooms':'int', 'sqft':'int', 'valuation':'int', 'yearbuilt':'int','lotsize':'int','pools':'int','quality':'int'})

In [ ]:
train_qual_scale, validate_qual_scale, test_qual_scale = scale_zillow(train_qual, validate_qual, test_qual)

In [ ]:
X_train = train_qual_scale.drop(columns=['valuation'])
y_train = train_qual_scale.valuation

X_validate = validate_qual_scale.drop(columns=['valuation'])
y_validate = validate_qual_scale.valuation

X_test = test_qual_scale.drop(columns=['valuation'])
y_test = test_qual_scale.valuation

In [ ]:
X_train

In [ ]:
X_train = X_train.drop(columns=['fips_name','fips_name_Los Angeles'])
X_validate = X_validate.drop(columns=['fips_name','fips_name_Los Angeles'])
X_test = X_test.drop(columns=['fips_name','fips_name_Los Angeles'])

In [ ]:
for i in range(1,7):
# initialize the ML algorithm
    lm = LinearRegression()

    # create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
    rfe = RFE(lm, n_features_to_select=i)

    # fit the data using RFE
    rfe.fit(X_train,y_train)  

    # get the mask of the columns selected
    feature_mask = rfe.support_

    # get list of the column names. 
    rfe_feature = X_train.iloc[:,feature_mask].columns.tolist()
    print(rfe_feature)

rfecv = RFECV(lm, min_features_to_select = )
rfecv.fit(X_train, y_train)
feature_mask = rfecv.support_
rfecv_feature = X_train.iloc[:,feature_mask].columns.tolist()
print(rfecv_feature)

In [ ]:
# We need y_train and y_validate to be dataframes to append the new columns with predicted values. 
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)

# 1. Predict target_pred_mean
valuation_pred_mean = y_train['valuation'].mean()
y_train['valuation_pred_mean'] = valuation_pred_mean
y_validate['valuation_pred_mean'] = valuation_pred_mean

# 2. compute target_pred_median
valuation_pred_median = y_train['valuation'].median()
y_train['valuation_pred_median'] = valuation_pred_median
y_validate['valuation_pred_median'] = valuation_pred_median

# 3. RMSE of target_pred_mean
rmse_train_mean = mean_squared_error(y_train.valuation, y_train.valuation_pred_mean)**(1/2)
rmse_validate_mean = mean_squared_error(y_validate.valuation, y_validate.valuation_pred_mean)**(1/2)

print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train_mean, 5), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate_mean, 5))

# 4. RMSE of G3_pred_median
rmse_train_median = mean_squared_error(y_train.valuation, y_train.valuation_pred_median)**(1/2)
rmse_validate_median = mean_squared_error(y_validate.valuation, y_validate.valuation_pred_median)**(1/2)

print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train_median, 5), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate_median, 5))

In [ ]:
# create the model object
lm = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm.fit(X_train, y_train.valuation)

# predict train
y_train['valuation_pred_lm'] = lm.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.valuation, y_train.valuation_pred_lm)**(1/2)

# predict validate
y_validate['valuation_pred_lm'] = lm.predict(X_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.valuation, y_validate.valuation_pred_lm)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
# create the model object
lars = LassoLars(alpha=2.0)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lars.fit(X_train, y_train.valuation)

# predict train
y_train['valuation_pred_lars'] = lars.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.valuation, y_train.valuation_pred_lars)**(1/2)

# predict validate
y_validate['valuation_pred_lars'] = lars.predict(X_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.valuation, y_validate.valuation_pred_lars)**(1/2)

print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
# create the model object
glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
glm.fit(X_train, y_train.valuation)

# predict train
y_train['valuation_pred_glm'] = glm.predict(X_train)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.valuation, y_train.valuation_pred_glm)**(1/2)

# predict validate
y_validate['valuation_pred_glm'] = glm.predict(X_validate)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.valuation, y_validate.valuation_pred_glm)**(1/2)

print("RMSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

In [ ]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree=2)

# fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(X_train)

# transform X_validate_scaled & X_test_scaled
X_validate_degree2 = pf.transform(X_validate)
X_test_degree2 = pf.transform(X_test)

In [ ]:
# create the model object
lm2 = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm2.fit(X_train_degree2, y_train.valuation)

# predict train
y_train['valuation_pred_lm2'] = lm2.predict(X_train_degree2)

# evaluate: rmse
rmse_train = mean_squared_error(y_train.valuation, y_train.valuation_pred_lm2)**(1/2)

# predict validate
y_validate['valuation_pred_lm2'] = lm2.predict(X_validate_degree2)

# evaluate: rmse
rmse_validate = mean_squared_error(y_validate.valuation, y_validate.valuation_pred_lm2)**(1/2)

print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)